In [1]:
import numpy as np
import cv2
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
train_dir = 'Datasets/train'
val_dir = 'Datasets/test'
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

In [3]:
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(48,48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 28709 images belonging to 7 classes.


In [4]:
validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(48,48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 7178 images belonging to 7 classes.


In [5]:
emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))
# emotion_model.load_weights('emotion_model.h5')

c:\Users\praka\New_Project\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
cv2.ocl.setUseOpenCL(False)

emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

In [7]:
emotion_model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=0.0001, decay=1e-6),
    metrics=['accuracy']
)


c:\Users\praka\New_Project\.venv\Lib\site-packages\keras\src\optimizers\base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [18]:
emotion_model_info = emotion_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=5,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/5
448/448 ━━━━━━━━━━━━━━━━━━━━ 103s 229ms/step - accuracy: 0.5152 - loss: 1.2785 - val_accuracy: 0.5266 - val_loss: 1.2437
Epoch 2/5
448/448 ━━━━━━━━━━━━━━━━━━━━ 10s 21ms/step - accuracy: 0.5469 - loss: 1.2255 - val_accuracy: 0.5265 - val_loss: 1.2440
Epoch 3/5
448/448 ━━━━━━━━━━━━━━━━━━━━ 122s 273ms/step - accuracy: 0.5391 - loss: 1.2319 - val_accuracy: 0.5421 - val_loss: 1.2276
Epoch 4/5
448/448 ━━━━━━━━━━━━━━━━━━━━ 7s 16ms/step - accuracy: 0.5000 - loss: 1.2906 - val_accuracy: 0.5420 - val_loss: 1.2244
Epoch 5/5
448/448 ━━━━━━━━━━━━━━━━━━━━ 95s 212ms/step - accuracy: 0.5516 - loss: 1.2004 - val_accuracy: 0.5372 - val_loss: 1.2081


In [20]:
emotion_model.save_weights('emotion_model.weights.h5')


In [21]:
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

In [22]:
cv2.ocl.setUseOpenCL(False)
cap = cv2.VideoCapture(0)

In [25]:
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break

    # Load Haar Cascade for face detection
    bounding_box = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    num_faces = bounding_box.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y - 50), (x + w, y + h + 10), (255, 0, 0), 2)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)

        # Debugging: Check input image properties
        print("Cropped Image Shape:", cropped_img.shape)
        print("Pixel Range:", np.min(cropped_img), "to", np.max(cropped_img))

        # Predict emotion
        emotion_prediction = emotion_model.predict(cropped_img)
        print("Prediction Probabilities:", emotion_prediction)

        maxindex = int(np.argmax(emotion_prediction))
        print("Predicted Emotion Index:", maxindex)

        # Display emotion label on video
        cv2.putText(frame, emotion_dict[maxindex], (x + 20, y - 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    # Display the resulting frame
    cv2.imshow('Video', cv2.resize(frame, (1200, 860), interpolation=cv2.INTER_CUBIC))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

Cropped Image Shape: (1, 48, 48, 1)
Pixel Range: 14 to 206
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Prediction Probabilities: [[0. 0. 1. 0. 0. 0. 0.]]
Predicted Emotion Index: 2
Cropped Image Shape: (1, 48, 48, 1)
Pixel Range: 9 to 212
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Prediction Probabilities: [[0.0000000e+00 0.0000000e+00 1.0000000e+00 0.0000000e+00 6.0019856e-25
  0.0000000e+00 0.0000000e+00]]
Predicted Emotion Index: 2
Cropped Image Shape: (1, 48, 48, 1)
Pixel Range: 8 to 204
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
Prediction Probabilities: [[5.918898e-27 0.000000e+00 1.000000e+00 0.000000e+00 0.000000e+00
  0.000000e+00 0.000000e+00]]
Predicted Emotion Index: 2
Cropped Image Shape: (1, 48, 48, 1)
Pixel Range: 8 to 226
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
Prediction Probabilities: [[0.0000000e+00 0.0000000e+00 3.2690734e-13 0.0000000e+00 1.0000000e+00
  0.0000000e+00 0.0000000e+00]]
Predicted Emotion Index: 4
Cropped Image Shape: (1, 48, 48, 1)
Pixel Range: 9 to 201
1/1 ━━━━━━━━━━━

KeyboardInterrupt: 